# Prepairing chesscom data to use

## Import and functions

In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [2]:
# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1
    
# retunr classoc Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    return 1/(1+np.power(10, (rb-rw)/400))


# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    game = parser.parse(text, actions=pgn.Actions())
    
    try:
        score=game.tag_pairs['Result']
    except:
        score='Unknown'  
    try:
        date=game.tag_pairs['Date']
    except:
        date='Unknown'
    try:
        time=game.tag_pairs['UTCTime']
    except:
        time='Unknown'
    try:
        eco=game.tag_pairs['ECO']
    except:
        eco='Unknown'
    try:
        ecourl=game.tag_pairs['ECOUrl']
    except:
        ecourl='Unknown'
    return {'score':score, 
            'date': date, 
            'time':time, 
            'ECO':eco, 
            'ECO_url':ecourl}

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val


In [3]:
# get json form site and return string
# exmple get_api_data_to_str('eric', '2014', '01')
def get_api_data_to_str(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
    if response.status_code ==429:
        with open('data/429.txt', 'w+') as f:
            f.write('variant, ')
            f.write(st)
            f.close()
    return st

# convert api data to pandas for next working
def get_str_data_to_pandas(strng):
    import json
    js = json.loads(strng)
    df=pd.DataFrame(js)
    
    return df

# convert multilevel json to table
def prepair_pandas_multilevel_data(df):
    num=len(df)
    new_df=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(df.values[i])
        new_df=pd.concat([new_df, b])
        
    return new_df 

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(game_stat)
    
    # cut long and unusefull columns
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)
    
    # pgn - long string
    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    # result - string like '1-0'
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
    # propability from classic formula
    game_stat['white_elo_forecast']=game_stat[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    game_stat['black_elo_forecast']=game_stat[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
        
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    return game_stat
    


In [20]:
# main function
def get_btch(player, year, month):
    import datetime
    
    # API -> string
    
    delay_lst=[0, 1, 4]
    # we give 3 attemps to get data
    for cs in range(3):
        try:
            data_string=get_api_data_to_str(player, year, month)
        except Exception:
            print('Error: appempt num', cs)
            # wait 0, 1 or 2 seconds after next attempt
            time.sleep(delay_lst[cs])
            if cs==2:
                print('No load data')
    
    # string -> json -> DataFrame
    multidata=get_str_data_to_pandas(data_string)
    
    # DataFrame -> use json normalise -> good df
    usefull_data=prepair_pandas_multilevel_data(multidata)
    
    # game prepair
    workng_data=get_pl_stat(usefull_data)
    
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    workng_data.to_csv('source/'+player+'_'+year+'_'+month+'_'+now+'.csv')
    
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(now, ': btch '+player+'_'+year+'_'+month+' finished')

In [5]:
# start getting with threading magic
def user_parse(user_list, year_range):   
    import threading
    # status counter
    i=0
    for user in user_list:
        for year in year_range:
            for month in range(1,13):
                t = threading.Thread(target=get_btch, args=(user, str(year), get_number(month)))
                t.start()
                i=i+1
    print('All btch', i,  'are started')   

In [6]:
# read all json in folder and concate DataFrame
def read_files(path):
    # reading
    l=[]
    cntr=0
    import os
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                 l.append(os.path.join(root, file))
            cntr=cntr+1
    print('Total', cntr, 'files founded')

    # merging
    i=0
    df=pd.DataFrame()
    for link in l:
        try:
            dfb=pd.read_csv(link)
        except:
            dfb=pd.DataFrame()
        df=pd.concat([df, dfb])
        
    return df

In [7]:
# data for predict current user
def learn_prepair(df, target_user):
    df['target_user']=target_user
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    df['date'] = df['date'].astype('datetime64[ns]')
    
    return df

In [8]:
# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()


## Start working

In [9]:
# collect players by countries
countries_list=['RU']
get_county_players_list(countries_list[0])

#user_list=pd.read_json('player_lists/players_list_RU_2019-08-11 17:57:25.json').sample(20)['players'].values
user_list=['andreyvict', 'sever043', 'Rosolimo']

In [10]:
user_list[0:30]

['andreyvict', 'sever043', 'Rosolimo']

In [11]:
year_range=range(2012, 2020)
user_parse(user_list, year_range)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/panda

Exception in thread Thread-27:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/panda

Exception in thread Thread-44:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/panda

Exception in thread Thread-60:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/panda

...

Exception in thread Thread-128:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pand

404
{"message":"Date cannot be set in the future","code":0}



Exception in thread Thread-161:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pan

Btch andreyvict_2018_06 finished
All btch 288 are started


Exception in thread Thread-197:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6bd45bc99515>", line 26, in get_btch
    workng_data=get_pl_stat(usefull_data)
  File "<ipython-input-3-901706374060>", line 64, in get_pl_stat
    'url']]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2934, in __getitem__
    raise_missing=True)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1354, in _convert_to_indexer
    return self._get_listlike_indexer(obj, axis, **kwargs)[1]
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py", line 1161, in _get_listlike_indexer
    raise_missing=raise_missing)
  File "/home/roman/anaconda3/lib/python3.7/site-packages/pand

In [21]:
df=read_files('source/')

Total 117 files founded


In [22]:
df.sample(6)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
25,0,chess,blitz,300,True,1,0-1,2015-02-06,21:05:48,2015-2,Rosolimo,Tomines,1499,1547,timeout,win,0.0,1.0,0.431359,0.568641,D08,https://www.chess.com/openings/D08-Queens-Gambit-Declined-Albin-Countergambit-3.dxe5-d4,https://www.chess.com/live/game/1051938207,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/tomines
68,0,chess,blitz,300,True,1,0-1,2014-08-07,20:33:54,2014-8,Rosolimo,TIME_PASSCHESS,1495,1464,resigned,win,0.0,1.0,0.544495,0.455505,D32,https://www.chess.com/openings/D32-Tarrasch-Defense-Two-Knights-Variation,https://www.chess.com/live/game/881166587,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/time_passchess
67,0,chess,blitz,300,True,1,1-0,2016-02-05,07:56:31,2016-2,Rosolimo,ronljudd,1519,1477,win,resigned,1.0,0.0,0.560150,0.439850,D20,https://www.chess.com/openings/D20-Queens-Gambit-Accepted-Old-Variation,https://www.chess.com/live/game/1449658313,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/ronljudd
122,0,chess,blitz,300,True,1,1/2-1/2,2013-12-08,11:42:31,2013-12,pacer1,Rosolimo,1569,1529,repetition,repetition,0.5,0.5,0.557312,0.442688,B10,https://www.chess.com/openings/B10-Caro-Kann-Defense,https://www.chess.com/live/game/666538478,https://api.chess.com/pub/player/pacer1,https://api.chess.com/pub/player/rosolimo
85,0,chess,blitz,300,True,1,1-0,2013-12-05,21:49:37,2013-12,mfl2013,Rosolimo,1540,1530,win,checkmated,1.0,0.0,0.514387,0.485613,B12,https://www.chess.com/openings/B12-Caro-Kann-Defense-Advance-Variation-3...Bf5-4.Bd3,https://www.chess.com/live/game/664333141,https://api.chess.com/pub/player/mfl2013,https://api.chess.com/pub/player/rosolimo
79,0,chess,blitz,300,True,1,0-1,2014-02-12,10:28:09,2014-2,Rosolimo,vukmilan,1533,1528,resigned,win,0.0,1.0,0.507195,0.492805,A41,https://www.chess.com/openings/A41-Queens-Pawn-Opening-Anglo-Slav-Opening,https://www.chess.com/live/game/724784954,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/vukmilan


In [23]:
ds=learn_prepair(df, 'Rosolimo')
#ds=learn_prepair(df, 'sever043')

In [24]:
ds.sample(3)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color
10,0,chess,blitz,180+2,True,1,0-1,2018-10-02,20:38:08,2018-10,Rosolimo,milappa11,1642,1682,checkmated,win,0.0,1.0,0.442688,0.557312,D30,https://www.chess.com/openings/D30-Queens-Gambit-Declined,https://www.chess.com/live/game/3115807782,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/milappa11,Rosolimo,0.0,1642,1682,0.442688,w
106,0,chess,blitz,180+2,True,1,0-1,2019-03-18,19:09:21,2019-3,shelter1,Rosolimo,1492,1479,timeout,win,0.0,1.0,0.518700,0.481300,E61,https://www.chess.com/openings/E61-Kings-Indian-Defense-Normal-Variation-4.Nf3,https://www.chess.com/live/game/3545267157,https://api.chess.com/pub/player/shelter1,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1479,1492,0.481300,b
9,0,chess,blitz,300,True,1,0-1,2018-07-07,10:49:34,2018-7,Andreyvict,BodyofChrist,994,1032,timeout,win,0.0,1.0,0.445531,0.554469,B12,https://www.chess.com/openings/B12-Caro-Kann-Defense-2.d4,https://www.chess.com/live/game/2924535404,https://api.chess.com/pub/player/andreyvict,https://api.chess.com/pub/player/bodyofchrist,Rosolimo,1.0,1032,994,0.554469,b


In [25]:
ds['year']=ds['date'].dt.year
ds['month']=ds['date'].dt.month

In [27]:
ds.groupby(['year', 'month']).sum()[['game', 'score']]#.plot()

game  score
year month             
2012 9        95   55.0
2013 9       215  105.5
     10      284  137.5
     11      287  146.0
     12      384  183.5
2014 1       355  182.5
     2       294  132.5
     3       262  127.0
     4       173   85.5
     5       202   89.5
     6       182   89.0
     7       222  112.0
     8       308  151.0
     9       149   70.5
     10       88   48.0
     11      193   88.0
     12      209  103.0
2015 1       147   72.0
     2       168   80.0
     3       189   96.0
     4       191   91.0
     5       222  106.0
     6        99   52.5
     7       349  172.5
     8       192   89.0
     9       258  119.0
     10      181   93.0
     11      185   87.0
     12      250  126.0
2016 1       247  117.0
     2       234  114.5
     3       109   49.0
     4       136   71.0
     5       130   62.5
     6        10    4.0
     7       113   58.0
     8       183   92.0
     9       114   50.5
     10      207  104.0
     11      116   52.0
     12      186   96.5
2017 1       158   82.0
     2        62   34.5
     3        94   41.5
     4       201  102.5
     5        56   24.5
     6        55   27.5
     7       124   66.0
     8       164   80.5
     9       140   77.0
     10      145   71.0
     11       71   40.5
     12       69   35.5
2018 1        98   41.5
     2        66   36.0
     3       146   78.5
     4       142   73.5
     5       219  112.0
     6       220  112.0
     7       183   85.5
     8       181   92.5
     9       266  128.0
     10      271  133.5
     11      332  167.0
     12      388  186.0
2019 1       226  103.5
     2       196   92.5
     3       330  169.0
     4       160   78.0
     5       112   57.5
     6       112   52.5
     7       151   73.5
     8        62   34.0

In [18]:
# tst=get_api_data_to_str('Rosolimo', '2015', '11')